In [369]:
import warnings
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

warnings.filterwarnings("ignore")
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 2000

In [370]:
general_news_dataset = pd.read_csv('general_news_dataset.csv')

(0, 5)

In [371]:
analyst_update_dataset = pd.read_csv('analyst_update_dataset.csv')
analyst_update_dataset[pd.isna(analyst_update_dataset.Ticker) == True].shape

(0, 5)

In [372]:
central_banks_dataset = pd.read_csv('central_banks_dataset.csv')
central_banks_dataset[pd.isna(central_banks_dataset['Data source']) == True].shape

(0, 5)

In [373]:
currencies_dataset = pd.read_csv('currencies_dataset.csv')
currencies_dataset[pd.isna(currencies_dataset['Ticker']) == True].shape

(0, 5)

In [374]:
dividend_dataset = pd.read_csv('dividend_dataset.csv')
dividend_dataset[pd.isna(dividend_dataset['Data source']) == True].shape

(0, 5)

In [375]:
earnings_dataset = pd.read_csv('earnings_dataset.csv')
earnings_dataset[pd.isna(earnings_dataset['Ticker']) == True].shape

(0, 5)

In [376]:
energy_oil_dataset = pd.read_csv('energy_oil_dataset.csv')
energy_oil_dataset[pd.isna(energy_oil_dataset['Ticker']) == True].shape

(0, 5)

In [377]:
financials_dataset = pd.read_csv('financials_dataset.csv')
financials_dataset[pd.isna(financials_dataset['Data source']) == True].shape

(0, 5)

In [378]:
gold_metals_dataset = pd.read_csv('gold_metals_dataset.csv')
gold_metals_dataset[pd.isna(gold_metals_dataset['Data source']) == True].shape

(0, 5)

In [379]:
investments_dataset = pd.read_csv('investments_dataset.csv')
investments_dataset[pd.isna(investments_dataset['Ticker']) == True].shape

(0, 5)

In [380]:
ipo_dataset = pd.read_csv('ipo_dataset.csv')
ipo_dataset[pd.isna(ipo_dataset['Ticker']) == True].shape

(0, 5)

In [381]:
legal_dataset = pd.read_csv('legal_dataset.csv')
legal_dataset[pd.isna(legal_dataset['Data source']) == True].shape

(0, 5)

In [382]:
macro_dataset = pd.read_csv('macro_dataset.csv')
macro_dataset[pd.isna(macro_dataset['Data source']) == True].shape

(0, 5)

In [383]:
markets_dataset = pd.read_csv('markets_dataset.csv')
markets_dataset[pd.isna(markets_dataset['Ticker']) == True].shape

(0, 5)

In [384]:
personnel_change_dataset = pd.read_csv('personnel_change_dataset.csv')
personnel_change_dataset[pd.isna(personnel_change_dataset['Ticker']) == True].shape

(0, 5)

In [385]:
politics_dataset = pd.read_csv('politics_dataset.csv')
politics_dataset[pd.isna(politics_dataset['Data source']) == True].shape

(0, 5)

In [386]:
product_news_dataset = pd.read_csv('product_news_dataset.csv')
product_news_dataset[pd.isna(product_news_dataset['Data source']) == True].shape

(0, 5)

In [387]:
stock_commentary_dataset = pd.read_csv('stock_commentary_dataset.csv')
stock_commentary_dataset[pd.isna(stock_commentary_dataset['Ticker']) == True].shape

(0, 5)

In [388]:
stock_movement_dataset = pd.read_csv('stock_movement_dataset.csv')
stock_movement_dataset[pd.isna(stock_movement_dataset['Data source']) == True].shape

(0, 5)

In [389]:
treasuries_dataset = pd.read_csv('treasuries_dataset.csv')
treasuries_dataset[pd.isna(treasuries_dataset['Data source']) == True].shape

(0, 5)

In [390]:
#!g2.mig
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


2023-01-18 15:29:06.838372: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 15:29:09.639815: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 2629 MB memory:  -> device: 0, name: GRID A100X-1-5C MIG 1g.5gb, pci bus id: 0000:8c:00.0, compute capability: 8.0


In [391]:
#!g2.mig
import torch

if torch.cuda.is_available():    
   
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GRID A100X-1-5C MIG 1g.5gb


In [392]:
#!g2.mig
import torch
from torch.utils.data import TensorDataset, SequentialSampler, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import numpy as np
sentences = analyst_update_dataset.Headline.tolist()
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
inputs = tokenizer(sentences, return_tensors="pt", padding=True)
input_ids = inputs['input_ids']
attention_masks = inputs['attention_mask']

batch_size = 32
prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
finbert.to(device)

finbert.eval()
predictions = []
number = 0
for batch in prediction_dataloader:
    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_input_mask = batch

    with torch.no_grad():
     
        outputs = finbert(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask)
    for logit in outputs.logits:

      logits = logit
      predicted_class_id = logits.argmax().item()
      logits = logits.detach().cpu().numpy()
      predicted_class = finbert.config.id2label[predicted_class_id]
      predictions.append(predicted_class)

In [393]:
#!g2.mig
len(sentences)

7023

In [394]:
#!g2.mig
analyst_update_dataset.shape

(7023, 5)

In [395]:
#!g2.mig
len(predictions)

7023

In [396]:
#!g2.mig
def count_labels(label):
  counter = 0
  for prediction in predictions:
    if prediction == label:
      counter += 1
  return counter

In [397]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 1872
Negative 774
Neutral 4377


In [398]:
#!g2.mig
analyst_update_dataset['Sentiment'] = predictions

In [399]:
#!g2.mig
analyst_update_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment
0,GOOGL,Financial news,Citigroup Remains Bullish on Facebook,Shares of social media giant Facebook NASDAQ ...,2015-04-05,Positive
1,CACC,Financial news,Ally Financial s Ratings Upgraded By Moody s ...,Ally Financial s NYSE ALLY ratings have be...,2019-02-11,Positive
2,QCOM,Financial news,Northland Capital upgrades Qualcomm on 3D sens...,Northland Capital upgrades Qualcomm NASDAQ QC...,2017-09-15,Positive
3,BAC,Financial news,BofAML ups Nvidia price target to near Street ...,Bank of America Merrill Lynch NYSE BAC raise...,2017-09-18,Positive
4,AAPL,Financial news,Apple to gain unconditional EU approval for Sh...,By Foo Yun Chee BRUSSELS Reuters Apple O ...,2018-08-22,Positive


In [400]:
analyst_update_dataset[analyst_update_dataset.Sentiment == 'Positive'].shape

(1872, 6)

In [401]:
analyst_update_dataset[analyst_update_dataset.Sentiment == 'Negative'].shape

(774, 6)

In [402]:
analyst_update_dataset[analyst_update_dataset.Sentiment == 'Neutral'].shape

(4377, 6)

In [403]:
analyst_update_dataset['Topic'] = 'Analyst Update'
analyst_update_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,GOOGL,Financial news,Citigroup Remains Bullish on Facebook,Shares of social media giant Facebook NASDAQ ...,2015-04-05,Positive,Analyst Update
1,CACC,Financial news,Ally Financial s Ratings Upgraded By Moody s ...,Ally Financial s NYSE ALLY ratings have be...,2019-02-11,Positive,Analyst Update
2,QCOM,Financial news,Northland Capital upgrades Qualcomm on 3D sens...,Northland Capital upgrades Qualcomm NASDAQ QC...,2017-09-15,Positive,Analyst Update
3,BAC,Financial news,BofAML ups Nvidia price target to near Street ...,Bank of America Merrill Lynch NYSE BAC raise...,2017-09-18,Positive,Analyst Update
4,AAPL,Financial news,Apple to gain unconditional EU approval for Sh...,By Foo Yun Chee BRUSSELS Reuters Apple O ...,2018-08-22,Positive,Analyst Update


In [488]:
analyst_update_dataset.to_csv('analyst_update_dataset.csv')

In [404]:
#!g2.mig
sentences = central_banks_dataset.Headline.tolist()
inputs = tokenizer(sentences, return_tensors="pt", padding=True)
input_ids = inputs['input_ids']
attention_masks = inputs['attention_mask']

batch_size = 32
prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
finbert.to(device)
finbert.eval()
predictions = []
number = 0
for batch in prediction_dataloader:
    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_input_mask = batch

    with torch.no_grad():
     
        outputs = finbert(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask)
    for logit in outputs.logits:

      logits = logit
      predicted_class_id = logits.argmax().item()
      logits = logits.detach().cpu().numpy()
      predicted_class = finbert.config.id2label[predicted_class_id]
      predictions.append(predicted_class)
      number += 1

In [405]:
print(len(sentences))
print(len(predictions))

3149
3149


In [406]:
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 384
Negative 688
Neutral 2077


In [407]:
central_banks_dataset['Sentiment'] = predictions
central_banks_dataset['Topic'] = 'Fed | Central Banks'
central_banks_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,AAPL,Financial news,U S plan to repatriate billions will have lim...,HONG KONG Reuters A U S push to encourage...,2017-02-06,Neutral,Fed | Central Banks
1,SPGI,Financial news,Markets Wait To See If ECB Draghi Adds To View...,Economic Data DE Germany August GfK Consumer ...,2012-07-26,Neutral,Fed | Central Banks
2,BLK,Financial news,ECB extends deadline for binding bids for Ital...,MILAN Reuters The European Central Bank ha...,2019-04-11,Neutral,Fed | Central Banks
3,BAC,Financial news,After sprinting to aid rupiah Indonesia c ban...,By Maikel Jefriando and Fransiska Nangoy JAKAR...,2018-07-27,Negative,Fed | Central Banks
4,MS,Financial news,China to Unleash 108 Billion in Reserves Cut ...,Bloomberg China s central bank will cut t...,2018-06-24,Neutral,Fed | Central Banks


In [489]:
central_banks_dataset.to_csv('central_banks_dataset.csv')

In [408]:
#!g2.mig
def predict(data, device, finbert, tokenizer):
    sentences = data.Headline.tolist()
    inputs = tokenizer(sentences, return_tensors="pt", padding=True)
    input_ids = inputs['input_ids']
    attention_masks = inputs['attention_mask']
    batch_size = 32
    prediction_data = TensorDataset(input_ids, attention_masks)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
    finbert.to(device)
    finbert.eval()
    predictions = []
    number = 0
    for batch in prediction_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask = batch
        
        with torch.no_grad():
     
            outputs = finbert(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask)
        for logit in outputs.logits:

          logits = logit
          predicted_class_id = logits.argmax().item()
          logits = logits.detach().cpu().numpy()
          predicted_class = finbert.config.id2label[predicted_class_id]
          number+=1
          predictions.append(predicted_class)
    return predictions

In [409]:
#!g2.mig
predictions = predict(currencies_dataset, device, finbert, tokenizer)

In [410]:
#!g2.mig
len(predictions)

3547

In [411]:
#!g2.mig
currencies_dataset.shape

(3547, 5)

In [412]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 742
Negative 1133
Neutral 1672


In [413]:
#!g2.mig
currencies_dataset['Sentiment'] = predictions
currencies_dataset['Topic'] = 'Currencies'
currencies_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,NWSA,Financial news,USD Extends Losses Against Major Counterparts,The U S dollar extended losses against the ma...,2012-12-12,Negative,Currencies
1,NWSA,Financial news,Euro Moves Lower Amid ECB Speculations,Speculations that the European Central Bank E...,2013-01-08,Negative,Currencies
2,NWSA,Financial news,Euro Soared On LTRO News As JPY CAD AUD Dived,Euro finally broke out of recent range last we...,2013-01-28,Neutral,Currencies
3,CME,Financial news,GBP USD Maintaining Momentum And Pushing Towa...,Retail currency traders can t seem to make up ...,2017-05-18,Positive,Currencies
4,NWSA,Financial news,EUR USD Touches Four Week High At 1 31,The last 24 hours has seen the Euro experience...,2013-06-04,Neutral,Currencies


In [490]:
#!g2.mig
currencies_dataset.to_csv('currencies_dataset.csv')

In [414]:
#!g2.mig
predictions = predict(dividend_dataset, device, finbert, tokenizer)

In [415]:
#!g2.mig
len(predictions)

1208

In [416]:
#!g2.mig
dividend_dataset.shape

(1208, 5)

In [417]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 409
Negative 55
Neutral 744


In [418]:
#!g2.mig
dividend_dataset['Sentiment'] = predictions
dividend_dataset['Topic'] = 'Dividend'
dividend_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,BAC,Financial news,Bank of America Corporation Deposit Shs Perp P...,Bank of America Corporation Deposit Shs Perp P...,2020-01-17,Neutral,Dividend
1,AVB,Financial news,AvalonBay Communities declares 1 47 dividend,AvalonBay Communities NYSE AVB declares 1 4...,2018-01-31,Neutral,Dividend
2,SNV,Financial news,Synovus declares 0 15 dividend,Synovus NYSE SNV declares 0 15 share quarte...,2017-09-08,Neutral,Dividend
3,SPGI,Financial news,S P Global declares 0 41 dividend,S P Global NYSE SPGI declares 0 41 share qu...,2017-06-27,Neutral,Dividend
4,SO,Financial news,Southern declares 0 60 dividend,Southern NYSE SO declares 0 60 share quart...,2018-07-16,Neutral,Dividend


In [491]:
#!g2.mig
dividend_dataset.to_csv('dividend_dataset.csv')

In [419]:
#!g2.mig
predictions = predict(earnings_dataset, device, finbert, tokenizer)

In [420]:
#!g2.mig
print(len(predictions))
print(earnings_dataset.shape)

16595
(16595, 5)


In [421]:
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 4909
Negative 2108
Neutral 9578


In [422]:
#!g2.mig
earnings_dataset['Sentiment'] = predictions
earnings_dataset['Topic'] = 'Earnings'
earnings_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,PCAR,Financial news,Should You Sell PACCAR PCAR Before Earnings,Investors are always looking for stocks that a...,2017-10-23,Neutral,Earnings
1,AVB,Financial news,REITs To Watch For Earnings Early Next Week A...,The Q3 earnings season for real estate investm...,2016-10-20,Neutral,Earnings
2,AAPL,Financial news,Game On What to Expect from Nike s Earnings,All eyes will be on NYSE NKE after today s ...,2015-03-19,Neutral,Earnings
3,TTM,Financial news,Group 1 Automotive GPI Lags Q3 Earnings And ...,Share price of Group 1 Automotive Inc NYSE ...,2016-10-25,Neutral,Earnings
4,AAPL,Financial news,Will Revenue Growth Favor Papa John s PZZA Q...,Papa John s International Inc NASDAQ PZZA ...,2018-02-19,Neutral,Earnings


In [492]:
#!g2.mig
earnings_dataset.to_csv('earnings_dataset.csv')

In [423]:
#!g2.mig
predictions = predict(energy_oil_dataset, device, finbert, tokenizer)

In [424]:
#!g2.mig
len(predictions)

6434

In [425]:
#!g2.mig
energy_oil_dataset.shape

(6434, 5)

In [426]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 1605
Negative 2158
Neutral 2671


In [427]:
#!g2.mig
energy_oil_dataset['Sentiment'] = predictions
energy_oil_dataset['Topic'] = 'Energy | Oil'
energy_oil_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,BAC,Financial news,Guess What Happened The Last Time The Price Of...,There has only been one other time in history ...,2014-12-01,Negative,Energy | Oil
1,GS,Financial news,Oil Peeks Above 57 a Barrel Amid Conflicting ...,Bloomberg Crude rose after one of the mos...,2017-12-14,Negative,Energy | Oil
2,SLB,Financial news,Energy sector on track for worst one day drop ...,U S crude oil plunges 3 to 42 85 bbl hitti...,2017-06-20,Positive,Energy | Oil
3,STLD,Financial news,Steel Stocks Weigh On Markets,This morning many of the leading steel stocks...,2013-01-08,Negative,Energy | Oil
4,C,Financial news,Commodity Sector Outlook Got You Down Conside...,By Commodities have started to seriously bo...,2014-01-29,Negative,Energy | Oil


In [493]:
#!g2.mig
energy_oil_dataset.to_csv('energy_oil_dataset.csv')

In [428]:
#!g2.mig
predictions = predict(financials_dataset, device, finbert, tokenizer)

In [429]:
#!g2.mig
print(len(predictions))
print(financials_dataset.shape)

17857
(17857, 5)


In [430]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 11363
Negative 5662
Neutral 832


In [431]:
#!g2.mig
financials_dataset['Sentiment'] = predictions
financials_dataset['Topic'] = 'Financials'
financials_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,WMB,Financial news,Williams Companies Inc The WMB Q4 Earnin...,Williams Companies Inc The NYSE WMB came...,2019-02-13,Negative,Financials
1,APA,Financial news,Apache APA Q2 Earnings And Revenues Surpass ...,U S energy firm Apache Corporation NYSE APA...,2019-08-01,Positive,Financials
2,NWSA,Financial news,Volume Explodes On Accusations Of Accounting Sham,Ebix Inc NASDAQ EBIX is an international ...,2013-02-23,Negative,Financials
3,NOC,Financial news,Northrop ups guidance again,Diluted EPS increased 11 to 3 15 from 2 85 ...,2017-07-26,Neutral,Financials
4,TDC,Financial news,Teradata TDC Q1 Earnings Beat Estimates,Teradata TDC came out with quarterly earning...,2019-05-02,Positive,Financials


In [494]:
#!g2.mig
financials_dataset.to_csv('financials_dataset.csv')

In [432]:
#!g2.mig
predictions = predict(general_news_dataset, device, finbert, tokenizer)

In [433]:
#!g2.mig
print(len(predictions))
print(general_news_dataset.shape)

12468
(12468, 5)


In [434]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 1061
Negative 2506
Neutral 8901


In [435]:
#!g2.mig
general_news_dataset['Sentiment'] = predictions
general_news_dataset['Topic'] = 'General News | Opinion'
general_news_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,JPM,Financial news,Will The Zombie Apocalypse End The Real Estate...,We covered the investment wisdom of phantoms y...,2013-10-31,Neutral,General News | Opinion
1,WMB,Financial news,Jets down Darnold draw testy Browns in primetime,The New York Jets will look to avoid an 0 2 st...,2019-09-12,Neutral,General News | Opinion
2,BHC,Financial news,Ackman s fund falls 7 percent in March hurt b...,BOSTON Reuters Billionaire investor Willia...,2016-04-01,Neutral,General News | Opinion
3,GS,Financial news,John Whitehead former leader of Goldman Sachs...,By Doina Chiacu WASHINGTON Reuters John Wh...,2015-02-07,Neutral,General News | Opinion
4,ANF,Financial news,Bubbles Anyone,Investment bubbles are not hard to spot There...,2015-11-22,Neutral,General News | Opinion


In [495]:
#!g2.mig
general_news_dataset.to_csv('general_news_dataset.csv')

In [436]:
#!g2.mig
predictions = predict(gold_metals_dataset, device, finbert, tokenizer)

In [437]:
#!g2.mig
print(len(predictions))
print(gold_metals_dataset.shape)

2342
(2342, 5)


In [438]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 691
Negative 615
Neutral 1036


In [439]:
#!g2.mig
gold_metals_dataset['Sentiment'] = predictions
gold_metals_dataset['Topic'] = 'Gold | Metals | Materials'
gold_metals_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,WMT,Financial news,Dollar gains gold loses luster amid robust U ...,By Lewis Krauskopf NEW YORK Reuters The U ...,2016-10-06,Positive,Gold | Metals | Materials
1,CME,Financial news,Gold prices gained in Asia as Fed views on rat...,Investing com Gold gained in Asia on Tuesday...,2016-02-22,Neutral,Gold | Metals | Materials
2,NFLX,Financial news,Implied Volatility Surging For Energous WATT ...,Investors in Energous Corporation NASDAQ WAT...,2018-12-23,Neutral,Gold | Metals | Materials
3,AA,Financial news,Gold remains near 2 year high even as global ...,Investing com Gold closed virtually flat on...,2016-07-11,Positive,Gold | Metals | Materials
4,GM,Financial news,Platinum Price May Outperform Gold Silver Bas...,We are seeing some healthy profit taking in pr...,2012-10-21,Positive,Gold | Metals | Materials


In [496]:
#!g2.mig
gold_metals_dataset.to_csv('gold_metals_dataset.csv')

In [440]:
#!g2.mig
predictions = predict(investments_dataset, device, finbert, tokenizer)

In [441]:
#!g2.mig
print(len(predictions))
print(investments_dataset.shape)

8724
(8724, 5)


In [442]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 1076
Negative 318
Neutral 7330


In [443]:
#!g2.mig
investments_dataset['Sentiment'] = predictions
investments_dataset['Topic'] = 'M&A | Investments'
investments_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,IX,Financial news,UPDATE 5 Monex buys Orix unit to form No 2 Jap...,Deal priced at estimated 246 million\n Ai...,2009-10-27,Neutral,M&A | Investments
1,NBL,Financial news,Devon s Divestiture Goal Reached With Access P...,Devon Energy Corporation NYSE DVN announce...,2016-07-14,Neutral,M&A | Investments
2,IP,Financial news,International Paper To Buy Weyerhaeuser s Pulp...,Paper and packaging firm International Paper C...,2016-05-03,Neutral,M&A | Investments
3,VZ,Financial news,Verizon Looking To Cut 1 Billion Off Its Yaho...,On Friday it was reported that telecommunicat...,2016-10-07,Neutral,M&A | Investments
4,GS,Financial news,RBS Might Transfer Assets To Netherlands In No...,The Royal Bank of Scotland LON RBS NYSE RB...,2018-12-10,Neutral,M&A | Investments


In [497]:
#!g2.mig
investments_dataset.to_csv('investments_dataset.csv')

In [444]:
#!g2.mig
predictions = predict(ipo_dataset, device, finbert, tokenizer)

In [445]:
#!g2.mig
print(len(predictions))
print(ipo_dataset.shape)

833
(833, 5)


In [446]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 103
Negative 87
Neutral 643


In [447]:
#!g2.mig
ipo_dataset['Sentiment'] = predictions
ipo_dataset['Topic'] = 'IPO'
ipo_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,WBA,Financial news,Marijuana Banking IPOs Weed Votes Will Hem...,The much awaited Cannabis Banking Bill was fin...,2019-04-03,Positive,IPO
1,BIDU,Financial news,China s Aurora Mobile files for IPO of America...,Chinese app services provider Aurora Mobile Li...,2018-06-29,Neutral,IPO
2,BX,Financial news,Auto1 may consider IPO in future but no need f...,BERLIN Reuters German used car dealing pla...,2019-03-27,Neutral,IPO
3,INTC,Financial news,Corporate investors pile into electric vehicle...,By Paul Lienert DETROIT Reuters Corporate ...,2019-01-11,Neutral,IPO
4,EXPE,Financial news,Trivago IPO Are Tech and Travel Rising Again,In a bit of good news for the tech IPO market ...,2016-11-18,Neutral,IPO


In [498]:
#!g2.mig
ipo_dataset.to_csv('ipo_dataset.csv')

In [448]:
#!g2.mig
predictions = predict(legal_dataset, device, finbert, tokenizer)

In [449]:
#!g2.mig
print(len(predictions))
print(legal_dataset.shape)

7356
(7356, 5)


In [450]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 225
Negative 1740
Neutral 5391


In [451]:
#!g2.mig
legal_dataset['Sentiment'] = predictions
legal_dataset['Topic'] = 'Legal | Regulation'
legal_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,XRX,Financial news,Icahn Deason urge Xerox shareholders to oppos...,Reuters Xerox Corp s N XRX plan to sell ...,2018-02-13,Neutral,Legal | Regulation
1,MSFT,Financial news,Ex Autonomy CFO sentenced in U S to 5 years p...,By Jonathan Stempel\n Reuters The former ch...,2019-05-14,Neutral,Legal | Regulation
2,FDX,Financial news,UPDATE 1 EU probes new China postal law over t...,EU analysing if new China postal law breache...,2009-04-29,Neutral,Legal | Regulation
3,JPM,Financial news,FINRA fines JPMorgan 1 25 million for failure...,NEW YORK Reuters The Financial Industry Re...,2017-11-21,Negative,Legal | Regulation
4,NFLX,Financial news,Tech giants prep for legal fight against FCC s...,A day after the release of official text of th...,2018-01-05,Neutral,Legal | Regulation


In [499]:
#!g2.mig
legal_dataset.to_csv('legal_dataset.csv')

In [452]:
#!g2.mig
predictions = predict(macro_dataset, device, finbert, tokenizer)

In [453]:
#!g2.mig
print(len(predictions))
print(macro_dataset.shape)

11210
(11210, 5)


In [454]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 2582
Negative 3565
Neutral 5063


In [455]:
#!g2.mig
macro_dataset['Sentiment'] = predictions
macro_dataset['Topic'] = 'Macro'
macro_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,MS,Financial news,11 Reasons Why U S Economic Growth Is The Wor...,Those that were predicting that the U S econo...,2017-04-30,Negative,Macro
1,DE,Financial news,U S farmers tighten belts to compete with che...,By Mark Weinraub and P J Huffstutter\nCHICAGO...,2017-10-26,Neutral,Macro
2,C,Financial news,LEI Index Points To Improving Economy But Is I...,Economic numbers out of the US continue to sur...,2012-04-20,Positive,Macro
3,NWSA,Financial news,World Competitiveness Rankings Europeans Lost...,As calls grow for Europe to ease up on its aus...,2013-05-30,Negative,Macro
4,BA,Financial news,U S Russia suspend nuclear treaty,The U S and Russia suspended a 1987 nuclear w...,2019-02-03,Neutral,Macro


In [500]:
#!g2.mig
macro_dataset.to_csv('macro_dataset.csv')

In [456]:
#!g2.mig
predictions = predict(markets_dataset, device, finbert, tokenizer)

In [457]:
#!g2.mig
print(len(predictions))
print(markets_dataset.shape)

24264
(24264, 5)


In [333]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 7984
Negative 7472
Neutral 9540


In [458]:
#!g2.mig
markets_dataset['Sentiment'] = predictions
markets_dataset['Topic'] = 'Markets'
markets_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,NWSA,Financial news,Street Just Yawns At Europe s Latest Crisis,Goldman Sachs called the Cyprus bank bailout a...,2013-03-19,Neutral,Markets
1,GPAGF,Financial news,Mexico stocks higher at close of trade IPC up...,Investing com Mexico stocks were higher afte...,2015-08-31,Positive,Markets
2,TGT,Financial news,FTSE up 0 9 percent early financials gain,Banks gain offsetting weakness in miners\n ...,2009-07-29,Positive,Markets
3,AMZN,Financial news,Markets Mixed On Last Week s Finish,US equities finished a choppy session and week...,2017-06-18,Neutral,Markets
4,GS,Financial news,U S stocks mixed after choppy session Dow do...,Investing com U S stocks finished Thursday ...,2014-10-16,Negative,Markets


In [501]:
#!g2.mig
markets_dataset.to_csv('markets_dataset.csv')

In [459]:
#!g2.mig
predictions = predict(personnel_change_dataset, device, finbert, tokenizer)

In [460]:
#!g2.mig
print(len(predictions))
print(personnel_change_dataset.shape)

2690
(2690, 5)


In [461]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 109
Negative 153
Neutral 2428


In [462]:
#!g2.mig
personnel_change_dataset['Sentiment'] = predictions
personnel_change_dataset['Topic'] = 'Personnel Change'
personnel_change_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,CSCO,Financial news,BlackBerry names Cisco veteran as global head ...,By Euan Rocha TORONTO Reuters BlackBerry L...,2015-07-13,Neutral,Personnel Change
1,KO,Financial news,Coca Cola names Quincey president fuelling su...,By Anjali Athavaley Reuters Coca Cola Co ...,2015-08-13,Neutral,Personnel Change
2,AAPL,Financial news,Qualcomm to depose Apple services chief Eddy Cue,By Stephen Nellis Reuters Qualcomm NASDAQ...,2018-05-04,Neutral,Personnel Change
3,GOOGL,Financial news,Google Canada head exits company names interi...,TORONTO Reuters Alphabet NASDAQ GOOGL In...,2017-07-25,Neutral,Personnel Change
4,BAC,Financial news,Exclusive Bank of America to hire 50 bankers ...,By Sumeet Chatterjee and Scott Murdoch HONG KO...,2020-01-22,Neutral,Personnel Change


In [502]:
#!g2.mig
personnel_change_dataset.to_csv('personnel_change_dataset.csv')

In [463]:
#!g2.mig
predictions = predict(politics_dataset, device, finbert, tokenizer)

In [464]:
#!g2.mig
print(len(predictions))
print(politics_dataset.shape)

4619
(4619, 5)


In [465]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 303
Negative 641
Neutral 3675


In [466]:
#!g2.mig
politics_dataset['Sentiment'] = predictions
politics_dataset['Topic'] = 'Politics'
politics_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,AAPL,Financial news,Crowd chants Modi Modi as India PM winds up...,By Yasmeen Abutaleb SAN JOSE California Reut...,2015-09-28,Neutral,Politics
1,AAPL,Financial news,Who Benefits From Trump s China Tariffs,Today we have another Trump success story t...,2019-07-15,Positive,Politics
2,AGN,Financial news,President Trump at it again with comments on ...,Drug and biotech stocks are again the target...,2017-10-16,Neutral,Politics
3,AMZN,Financial news,Factbox India s Modi crafts populist measures...,By Krishna N Das NEW DELHI Reuters India ...,2019-01-25,Neutral,Politics
4,GOOGL,Financial news,IForex Daily Looking Ahead To FOMC Speech,The dollar turned broadly higher against the e...,2015-03-25,Neutral,Politics


In [503]:
#!g2.mig
politics_dataset.to_csv('politics_dataset.csv')

In [467]:
#!g2.mig
predictions = predict(product_news_dataset, device, finbert, tokenizer)

In [468]:
#!g2.mig
print(len(predictions))
print(product_news_dataset.shape)

44401
(44401, 5)


In [469]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 8893
Negative 4275
Neutral 31233


In [470]:
#!g2.mig
product_news_dataset['Sentiment'] = predictions
product_news_dataset['Topic'] = 'Company | Product News'
product_news_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,GM,Financial news,GM to respond by January 7 to Canada union pro...,Reuters General Motors Co NYSE GM will r...,2018-12-20,Neutral,Company | Product News
1,WFC,Financial news,Wells Fargo plans 1 000 U S job cuts,By Imani Moise Reuters Wells Fargo NYSE...,2018-11-15,Neutral,Company | Product News
2,ROK,Financial news,ManpowerGroup Collaborates With Rockwell To Tr...,Leading staffing company ManpowerGroup Inc ...,2017-06-13,Neutral,Company | Product News
3,SRPT,Financial news,Sarepta Looks To Focus On Exondys 51 DMD Pip...,We issued an updated research report on Cambri...,2017-06-29,Neutral,Company | Product News
4,AAPL,Financial news,HSBC retail head sees more bank partnerships w...,TORONTO Reuters HSBC s L HSBA retail ban...,2016-09-12,Neutral,Company | Product News


In [504]:
#!g2.mig
product_news_dataset.to_csv('product_news_dataset.csv')

In [471]:
#!g2.mig
stock_commentary_dataset.shape

(24364, 5)

In [472]:
#!g2.mig
def predict(sentences, device, finbert, tokenizer):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True)
    input_ids = inputs['input_ids']
    attention_masks = inputs['attention_mask']
    #labels = torch.tensor(labels)
    # Set the batch size.
    batch_size = 32
    prediction_data = TensorDataset(input_ids, attention_masks)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
    finbert.to(device)
    # Put model in evaluation mode
    finbert.eval()
    predictions = []
    number = 0
    for batch in prediction_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask = batch

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up prediction
        with torch.no_grad():
            # Forward pass, calculate logit predictions
     
            outputs = finbert(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask)
        for logit in outputs.logits:

          logits = logit
          predicted_class_id = logits.argmax().item()
          # Move logits and labels to CPU
          logits = logits.detach().cpu().numpy()
          # Store predictions and true labels
          predicted_class = finbert.config.id2label[predicted_class_id]
          number+=1
          predictions.append(predicted_class)
    return predictions

In [474]:
#!g2.mig
sentences = stock_commentary_dataset.Headline.tolist()
count = 0
tokenize_sentences = []
predictions = []
for sent in sentences:
    count += 1
    tokenize_sentences.append(sent)
    if count >= 1000:
        predictions.extend(predict(tokenize_sentences, device, finbert, tokenizer))
        tokenize_sentences = []
        count = 0
    else:
        if len(predictions) == 24000 and count == 364:
            predictions.extend(predict(tokenize_sentences, device, finbert, tokenizer))
            tokenize_sentences = []
            count = 0

In [475]:
#!g2.mig
#print(len(predictions))
print(stock_commentary_dataset.shape)
print(len(predictions))

(24364, 5)
24364


In [476]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 9650
Negative 1634
Neutral 13080


In [477]:
#!g2.mig
stock_commentary_dataset['Sentiment'] = predictions
stock_commentary_dataset['Topic'] = 'Stock Commentary'
stock_commentary_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,CATM,Financial news,Here s Why You Should Hold Alliance Data ADS ...,Alliance Data Systems Corporation NYSE ADS ...,2018-12-17,Neutral,Stock Commentary
1,LHX,Financial news,What Makes Harris HRS A Strong Momentum Stoc...,Momentum investing is all about the idea of fo...,2019-02-12,Positive,Stock Commentary
2,PFG,Financial news,Are You Looking For A High Growth Dividend Sto...,Whether it s through stocks bonds ETFs or o...,2019-06-19,Positive,Stock Commentary
3,JBLU,Financial news,Here s Where I ll Buy JetBlue,JetBlue Airways NASDAQ JBLU has been steadil...,2017-08-21,Neutral,Stock Commentary
4,LAZ,Financial news,Why Regions Financial RF Is A Great Pick Rig...,Shares of Birmingham based banking giant Regi...,2016-11-02,Positive,Stock Commentary


In [505]:
#!g2.mig
stock_commentary_dataset.to_csv('stock_commentary_dataset.csv')

In [478]:
#!g2.mig
sentences  = stock_movement_dataset.Headline.tolist()
predictions = predict(sentences, device, finbert, tokenizer)

In [479]:
#!g2.mig
print(len(predictions))
print(stock_movement_dataset.shape)

19915
(19915, 5)


In [480]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 8862
Negative 5493
Neutral 5560


In [481]:
#!g2.mig
stock_movement_dataset['Sentiment'] = predictions
stock_movement_dataset['Topic'] = 'Stock Movement'
stock_movement_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,GE,Financial news,General Electric Shares Jump Despite Lower Tha...,Investing com Shares of General Electric ...,2018-01-24,Positive,Stock Movement
1,GOOGL,Financial news,Bitcoin rockets above 5 000 to all time high,By Jemima Kelly LONDON Reuters Bitcoin sma...,2017-10-12,Positive,Stock Movement
2,AXP,Financial news,Visa Or Mastercard Which Stock Looks Better P...,Visa Inc NYSE V and Mastercard Inc NYS...,2019-05-15,Positive,Stock Movement
3,TM,Financial news,Japanese Car Makers Rally On Weak Yen Hyundai...,The beleaguered Japanese auto shares are rally...,2012-11-18,Positive,Stock Movement
4,BA,Financial news,Esterline Technologies ESL Soars To 52 Week ...,Have you been paying attention to shares of Es...,2018-12-27,Neutral,Stock Movement


In [506]:
#!g2.mig
stock_movement_dataset.to_csv('stock_movement_dataset.csv')

In [482]:
#!g2.mig
sentences = treasuries_dataset.Headline.tolist()
predictions = predict(sentences, device, finbert, tokenizer)

In [483]:
#!g2.mig
print(len(predictions))
print(treasuries_dataset.shape)

2514
(2514, 5)


In [484]:
#!g2.mig
print('Positive', count_labels('Positive'))
print('Negative', count_labels('Negative'))
print('Neutral', count_labels('Neutral'))

Positive 335
Negative 521
Neutral 1658


In [485]:
#!g2.mig
treasuries_dataset['Sentiment'] = predictions
treasuries_dataset['Topic'] = 'Treasuries | Corporate Debt'
treasuries_dataset.head(5)

,Ticker,Data source,Headline,Content,Release date,Sentiment,Topic
0,AAPL,Financial news,Citigroup To Redeem Subordinated Notes Worth C...,Citigroup Inc NYSE C recently announced 1...,2017-09-10,Neutral,Treasuries | Corporate Debt
1,TSLA,Financial news,Tesla raises borrowing capacity for car leases...,Reuters Tesla Inc O TSLA said on Friday ...,2017-10-20,Positive,Treasuries | Corporate Debt
2,MS,Financial news,Week That Spooked Bond Markets Sent Traders Sc...,Bloomberg Volatility in the U S Treasury...,2018-10-12,Negative,Treasuries | Corporate Debt
3,BAC,Financial news,US 10 Year Yields Fall From 12 Month High,Treasuries advanced after 10 year U S governm...,2013-05-30,Negative,Treasuries | Corporate Debt
4,MMM,Financial news,Scandi FX Volatility Monitor Sell 3M EUR SEK...,Implied volatility is little changed from last...,2019-01-14,Neutral,Treasuries | Corporate Debt


In [507]:
#!g2.mig
treasuries_dataset.to_csv('treasuries_dataset.csv')